In [2]:
import pandas as pd
import seaborn as sns
import sys
import string
print('Original sys.path:', sys.path)

# Append a new directory to sys.path
sys.path.append('C:/Users/joepa/OneDrive/Desktop/computerScienceYear4/Data Mining/summary_eval/summary_eval')

# Print the updated sys.path
print('Updated sys.path:', sys.path)
from summary_eval.data import summary_df, prompts_df
from summary_eval.settings import TRAIN_SIZE
from summary_eval.testing import cross_validate

merged_df = pd.merge(summary_df, prompts_df, on='prompt_id')

Original sys.path: ['c:\\Users\\joepa\\OneDrive\\Desktop\\computerScienceYear4\\Data Mining\\summary_eval\\notebooks', 'C:\\Users\\joepa\\OneDrive\\Desktop\\computerScienceYear4\\Data Mining\\summary_eval', 'C:\\Users\\joepa\\anaconda3\\python39.zip', 'C:\\Users\\joepa\\anaconda3\\DLLs', 'C:\\Users\\joepa\\anaconda3\\lib', 'C:\\Users\\joepa\\anaconda3', 'c:\\Users\\joepa\\OneDrive\\Desktop\\computerScienceYear4\\Data Mining\\summary_eval\\venv', '', 'c:\\Users\\joepa\\OneDrive\\Desktop\\computerScienceYear4\\Data Mining\\summary_eval\\venv\\lib\\site-packages', 'c:\\Users\\joepa\\OneDrive\\Desktop\\computerScienceYear4\\Data Mining\\summary_eval\\venv\\lib\\site-packages\\win32', 'c:\\Users\\joepa\\OneDrive\\Desktop\\computerScienceYear4\\Data Mining\\summary_eval\\venv\\lib\\site-packages\\win32\\lib', 'c:\\Users\\joepa\\OneDrive\\Desktop\\computerScienceYear4\\Data Mining\\summary_eval\\venv\\lib\\site-packages\\Pythonwin']
Updated sys.path: ['c:\\Users\\joepa\\OneDrive\\Desktop\\com

2024-03-16 20:25:46,345 - INFO - Read 7165 summaries from C:\Users\joepa\OneDrive\Desktop\computerScienceYear4\Data Mining\summary_eval\data\summaries_train.csv
2024-03-16 20:25:46,365 - INFO - Read 4 prompts from C:\Users\joepa\OneDrive\Desktop\computerScienceYear4\Data Mining\summary_eval\data\prompts_train.csv


In [3]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Load pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Generate BERT embeddings
def generate_bert_embeddings(text):
    with torch.no_grad():
        tokens = tokenizer.encode(text, add_special_tokens=True)
        outputs = bert_model(torch.tensor([tokens]))
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()  # Average pooling over all tokens

# Apply function to 'text', 'prompt_text', and 'prompt_question' columns
merged_df['text_embeddings'] = merged_df['text'].apply(generate_bert_embeddings)
merged_df['prompt_embeddings'] = merged_df['prompt_text'].apply(generate_bert_embeddings)
merged_df['prompt_question_embeddings'] = merged_df['prompt_question'].apply(generate_bert_embeddings)

# Concatenate embeddings
X = np.concatenate([
    np.stack(merged_df['text_embeddings'].values),
    np.stack(merged_df['prompt_embeddings'].values),
    np.stack(merged_df['prompt_question_embeddings'].values)
], axis=1)

# Define targets
y = merged_df[['content_score', 'wording_score']]

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Choose model
model = LinearRegression()
# model = RandomForestRegressor()

# Train model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)


None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


ModuleNotFoundError: No module named 'torch'